<a href="https://colab.research.google.com/github/shreyansh273/Football-Scouting/blob/main/Player_stats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
import sys, getopt
import csv

In [ ]:
url = 'https://fbref.com/en/comps/9/stats/Premier-League-Stats'

In [ ]:
res = requests.get(url)
## The next two lines get around the issue with comments breaking the parsing.
comm = re.compile("<!--|-->")
soup = BeautifulSoup(comm.sub("",res.text),'lxml')

In [ ]:
all_tables = soup.findAll("tbody")
team_table = all_tables[0]
player_table = all_tables[1]

In [ ]:
pre_df_squad = dict()
#Note: features does not contain squad name, it requires special treatment
features_wanted_squad = {"players_used","games"}